In [1]:
import torch.nn as nn
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.auto import tqdm
import torchmetrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from copy import copy
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
from tsai.all import GRU_FCN
import os
import argparse
from urllib.parse import urlparse
from io import StringIO


device0 = torch.device("cpu")


class CustomDataset(Dataset):
    def __init__(self, data, transform=None, target_transform=None):
        self.PPG_data = data

    def __len__(self):
        return len(self.PPG_data)

    def __getitem__(self, idx):

        signal_tensor = self.PPG_data[idx]

        return signal_tensor


def test(model, dataloader):
    validation_tqdm = tqdm(dataloader)
    model.eval()
    val_count = 0
    predict_list = []
    with torch.no_grad():
        for x in validation_tqdm:
            val_count += 1
            x = x.to(device0).float()
            predict = model(x).to(device0)
            predict_list.append(F.sigmoid(predict).item())
    return predict_list